# KNOWLEDGE GRAPH

### 1. SETUP
To prepare your environment, you need to install some packages and enter credentials for the Watson services.



### 1.1 Install the necessary packages
You need the latest versions of these packages:
Watson Developer Cloud: a client library for Watson services.
NLTK: leading platform for building Python programs to work with human language data.


In [ ]:
!pip install mammoth

## Install the Watson Developer Cloud package:

In [ ]:
!pip install watson-developer-cloud==1.5

#### Install NLTK:

In [ ]:
!pip install --upgrade nltk

#### Install IBM Object Storage Client

In [ ]:
!pip install ibm-cos-sdk

### 1.2 Import packages and libraries
Import the packages and libraries that you'll use:

In [ ]:
import json
import pandas as pd
import mammoth
import os, re
import networkx as nx
import io
from io import StringIO
import matplotlib.pyplot as plt
from collections import Iterable 
from io import BytesIO
import zipfile

import ibm_boto3
from botocore.client import Config

import nltk
from nltk import word_tokenize,sent_tokenize,ne_chunk
from nltk.corpus import stopwords

from bs4 import BeautifulSoup
from bs4.element import Comment


from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, SemanticRolesOptions, RelationsOptions, KeywordsOptions

In [ ]:
nltk.download()

## 2. Configuration
Add configurable items of the notebook below



### 2.1 Add your service credentials from Bluemix for the Watson services
You must create a Watson Natural Language Understanding service on Bluemix. Create a service for Natural Language Understanding (NLU). Insert the username and password values for your NLU in the following cell. Do not change the values of the version fields.

Run the cell.

In [ ]:
natural_language_understanding = NaturalLanguageUnderstandingV1(
  username= '',
  password='',
  version='2017-02-27')

### 2.2 Add your service credentials for Object Storage


You must create Object Storage service on Bluemix. To access data in a file in Object Storage, you need the Object Storage authentication credentials. Insert the Object Storage authentication credentials as credentials_1 in the following cell after removing the current contents in the cell.

In [ ]:
#Insert the Object Storage authentication credentials as credentials_1 here

credentials_1 = {
    'IBM_API_KEY_ID': '',
    'IAM_SERVICE_ID': '',
    'ENDPOINT': '',
    'IBM_AUTH_ENDPOINT': '',
    'BUCKET': '',
}




In [ ]:
#Add credentials of config_classification.txt here
#Insert the authentication credentials as credentials_2 

In [ ]:
#Add credentials of config__relations.txt here
#Insert the authentication credentials as credentials_3 

In [ ]:
# Insert archive.zip as StreamingBody object 

## 3. Persistence and Storage


#### 3.1 Configure Object Storage Client

In [ ]:
cos = ibm_boto3.client('s3',
                    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials_1['ENDPOINT'])

def get_file(filename):
    '''Retrieve file from Cloud Object Storage'''
    fileobject = cos.get_object(Bucket=credentials_1['BUCKET'], Key=filename)['Body']
    return fileobject

    
def get_docx_file():
    '''Retrieve file '''
    docx_files=[]
    zip_ref = zipfile.ZipFile(BytesIO(streaming_body_5.read()),'r')
    paths = zip_ref.namelist()
    for path in paths:
        file=zip_ref.extract(path)
        docx_files.append(file)
    return docx_files



def load_string(fileobject):
    '''Load the file contents into a Python string'''

    text = fileobject.read()
    return text

def load_df(fileobject,sheetname):
    '''Load file contents into a Pandas dataframe'''
    excelFile = pd.ExcelFile(fileobject)
    df = excelFile.parse(sheetname)
    return df

def put_file(filename, filecontents):
    '''Write file to Cloud Object Storage'''
    resp = cos.put_object(Bucket=credentials_1['BUCKET'], Key=filename, Body=filecontents)
    return resp

### 4. Data Preparation 

#### 4.1 Global variables and functions.

In [ ]:
# Maintain tagged text and plain text map
tagTextMap ={}


    
def HtmlToJson(html_file):
    '''
    function to convert information in Html tables to Json
    '''
    html_list = pd.read_html(html_file)
    # ''' converting the list of dataframe into one dataframe '''
    dataframe = pd.concat(html_list)

    dataframe, dataframe.columns, dataframe.columns.name = dataframe.iloc[1:], dataframe.loc[0].astype(str), None
    
    j = dataframe.to_json(orient='records')
    info_json = json.loads(j)
    return info_json

def getRawTextandHtmlTableAsJson(docx_file):
    '''
    function to extract text and Html tables from docx to Json
    '''
    result = mammoth.convert_to_html(docx_file)
    html_table = HtmlToJson(result.value)
    raw_text = mammoth.extract_raw_text(docx_file)
    raw_text = raw_text.value
    messages = result.messages 
    return html_table,raw_text
    
def generate_NLUJson_HtmlTablesJson():
    '''
    function that returns augmented results from nlu, html tables, raw texts
    '''
    html_tables=[]
    raw_texts=[]
    docx_files=[]
    augmented_results_from_nlu=[]
    docx_files= get_docx_file()
    for docx_file in docx_files:
        with open(docx_file,'rb') as doc_file:
            html_table,raw_text = getRawTextandHtmlTableAsJson(doc_file)
            html_tables.append(html_table)
            raw_texts.append(raw_text)
    for raw_text in raw_texts:
        response_nlu = classify_text(str(raw_text), config_classification_json)
        augmented_results_from_nlu.append(response_nlu)

    return augmented_results_from_nlu, html_tables, raw_texts
    

### 5. Watson Text Classification
Write the classification related utility functions in a modularalized form.



#### 5.1 Watson NLU Classification 

In [ ]:
def analyze_using_NLU(text_content):
    '''
    Call Watson Natural Language Understanding service to obtain analysis results.
    '''
    response = natural_language_understanding.analyze(
        text= text_content,
        features=Features(
        entities=EntitiesOptions(),
        relations=RelationsOptions(),
        keywords= KeywordsOptions())
    )
    return response

#### 5.2 Augumented Classification
Custom classification utlity functions for augumenting the results of Watson NLU API call


In [ ]:
def split_sentences(text):
    """ Split text into sentences.
    """
    sentence_delimiters = re.compile(u'[\\[\\]\n.!?]')
    sentences = sentence_delimiters.split(text)
    return sentences

def split_into_tokens(text):
    """ Split text into tokens.
    """
    tokens = nltk.word_tokenize(text)
    return tokens
    
def POS_tagging(text):
    """ Generate Part of speech tagging of the text.
    """
    POSofText = nltk.tag.pos_tag(text)
    return POSofText

def keyword_tagging(tag,tagtext,text):
    """ Tag the text matching keywords.
    """
    if (text.lower().find(tagtext.lower()) != -1):
        return text[text.lower().find(tagtext.lower()):text.lower().find(tagtext.lower())+len(tagtext)]
    else:
        return 'UNKNOWN'
    
def regex_tagging(tag,regex,text):
    """ Tag the text matching REGEX.
    """    
    p = re.compile(regex, re.IGNORECASE)
    matchtext = p.findall(text)
    regex_list=[]    
    if (len(matchtext)>0):
        for regword in matchtext:
            regex_list.append(regword)
    return regex_list

def chunk_tagging(tag,chunk,text):
    """ Tag the text using chunking.
    """
    parsed_cp = nltk.RegexpParser(chunk)
    pos_cp = parsed_cp.parse(text)
    chunk_list=[]
    for root in pos_cp:
        if isinstance(root, nltk.tree.Tree):               
            if root.label() == tag:
                chunk_word = ''
                for child_root in root:
                    chunk_word = chunk_word +' '+ child_root[0]
                chunk_list.append(chunk_word)
    return chunk_list
    
def augument_NLUResponse(responsejson,updateType,text,tag):
    """ Update the NLU response JSON with augumented classifications.
    """
    if(updateType == 'keyword'):
        if not any(d.get('text', None) == text for d in responsejson['keywords']):
            responsejson['keywords'].append({"text":text,"relevance":0.5})
    else:
        if not any(d.get('text', None) == text for d in responsejson['entities']):
            responsejson['entities'].append({"type":tag,"text":text,"relevance":0.5,"count":1})        
    

def classify_text(text, config):
    """ Perform augumented classification of the text.
    """
    
    response = analyze_using_NLU(text)
    responsejson = response
    
    sentenceList = split_sentences(text)
    
    tokens = split_into_tokens(text)
    
    postags = POS_tagging(tokens)
    
    configjson = json.loads(config)
    
    for stages in configjson['configuration']['classification']['stages']:
        for steps in stages['steps']:
            if (steps['type'] == 'keywords'):
                for keyword in steps['keywords']:
                    for word in sentenceList:
                        wordtag = keyword_tagging(keyword['tag'],keyword['text'],word)
                        if(wordtag != 'UNKNOWN'):
                            augument_NLUResponse(responsejson,'entities',wordtag,keyword['tag'])
            elif(steps['type'] == 'd_regex'):
                for regex in steps['d_regex']:
                    for word in sentenceList:
                        regextags = regex_tagging(regex['tag'],regex['pattern'],word)
                        if (len(regextags)>0):
                            for words in regextags:
                                augument_NLUResponse(responsejson,'entities',words,regex['tag'])
            elif(steps['type'] == 'chunking'):
                for chunk in steps['chunk']:
                    chunktags = chunk_tagging(chunk['tag'],chunk['pattern'],postags)
                    if (len(chunktags)>0):
                        for words in chunktags:
                            augument_NLUResponse(responsejson,'entities',words,chunk['tag'])
            else:
                print('UNKNOWN STEP')
    
    return responsejson

def replace_unicode_strings(response):
    """ Convert dict with unicode strings to strings.
    """
    if isinstance(response, dict):
        return {replace_unicode_strings(key): replace_unicode_strings(value) for key, value in response.iteritems()}
    elif isinstance(response, list):
        return [replace_unicode_strings(element) for element in response]
    elif isinstance(response, unicode):
        return response.encode('utf-8')
    else:
        return response

#### 5.3 Correferencing and Augmented Relations.

In [ ]:
def chunk_sentence(text):
    """ Tag the sentence using chunking.
    """
    grammar = """
      NP: {<DT|JJ|PRP|NN.*>+} # Chunk sequences of DT,JJ,NN
          #}<VB*|DT|JJ|RB|PRP><NN.*>+{  # Chink sequences of VB,DT,JJ,NN       
      PP: {<IN><NP>}               # Chunk prepositions followed by NP
      V: {<V.*>}                   # Verb      
      VP: {<VB*><NP|PP|CLAUSE>+}  # Chunk verbs and their arguments
      CLAUSE: {<NP><VP>}           # Chunk NP, VP
      """  
    parsed_cp = nltk.RegexpParser(grammar,loop=2)
    pos_cp = parsed_cp.parse(text)
    return pos_cp
    
def find_attrs(subtree,phrase):
    attrs = ''
    if phrase == 'NP':
        for nodes in subtree:
            if nodes[1] in ['DT','PRP$','POS','JJ','CD','ADJP','QP','NP','NNP']:
                attrs = attrs+' '+nodes[0]
    return attrs    
    
def find_subject(t):
    for s in t.subtrees(lambda t: t.label() == 'NP'):
        return find_attrs(s,'NP')
    
def resolve_coreference(text, config):
    """ Resolve coreferences in the text for Nouns that are Subjects in a sentence
    """
    sentenceList = split_sentences(text)
    referenceSubject = ''
    sentenceText = ''
    configjson = json.loads(config)
    
    for sentences in sentenceList:    
        tokens = split_into_tokens(sentences)   
        postags = POS_tagging(tokens)
        sentencetags = chunk_sentence(postags)
        subjects = find_subject(sentencetags)
        for rules in configjson['configuration']['coreference']['rules']:
            if (rules['type'] == 'chunking'):
                for tags in rules['chunk']:
                    chunktags = chunk_tagging(tags['tag'],tags['pattern'],postags)
                    if (len(chunktags)>0):
                        for words in chunktags:
                            if tags['tag'] == 'PRP':
                                if subjects == '':
                                    sentenceText = sentenceText+sentences.replace(words,referenceSubject)+'. '
                            elif tags['tag'] == 'NAME':
                                if words == subjects:
                                    referenceSubject = words
                                    sentenceText = sentenceText+sentences+'. '
                    
    return sentenceText

def disambiguate_entities(text):
    """ Resolve disambiguity in the text using entities and entity resolution performed using Watson NLU
    """    
    sentenceList = split_sentences(text)
    taggedtext = text
    response = analyze_using_NLU(text)
    responsejson = response
    for sentences in sentenceList:
        tokens = split_into_tokens(sentences)
        postags = POS_tagging(tokens)
        name_tagged_text = chunk_tagging('NAME','NAME:{<NNP>+}',postags)
    for entities in responsejson['entities']:
        regexstr = entities['text']+'(?!>)'
        regex = re.compile(regexstr, re.IGNORECASE)
        tagText = '<'+entities['type']+':'+entities['text']+'>'
        taggedtext = re.sub(regexstr,tagText,taggedtext)
        tagTextMap[tagText] = entities['text']
    
    for roles in responsejson['semantic_roles']:
        if 'entities' not in roles['subject']:
            print('NO ENTITY')
        else:
            for entity in roles['subject']['entities']:
                if 'disambiguation' not in entity:
                    print('NO DISAMBIGUATION')
                else:
                    regexstr = roles['subject']['text']+'(?!>)'
                    regex = re.compile(regexstr, re.IGNORECASE)
                    tagText = '<'+entity['type']+':'+entity['text']+'>'
                    taggedtext = re.sub(regexstr,tagText,taggedtext)
                    tagTextMap[tagText] = entity['text']
    
    return taggedtext

def extract_relations(text, config,relations):
    """ Extract entity relationships in a sentence
    """    
    sentenceList = split_sentences(text)
    configjson = json.loads(config)
     
    for sentences in sentenceList:
        for rules in configjson['configuration']['relations']['rules']:
            if (rules['type'] == 'd_regex'):
                for regex in rules['d_regex']:
                    regextags = regex_tagging(regex['tag'],regex['pattern'],sentences)
                    if (len(regextags)>0):
                        for words in regextags:
                            relations.append((tagTextMap[words[0]],regex['tag'],tagTextMap[words[2]]))
         
    return relations

#### 5.4 Knowledge graph utility functions.

In [ ]:
def create_nodes_dataframe(G):
    '''
    function to create nodes dataframe
    '''
    nodes_df = pd.DataFrame(list(G.nodes(data=True)))
    nodes_df.columns = ['entity_names','entitity_attributes']
    return nodes_df

def create_entityNodes(G,results_from_nlu):
    '''
    function to create entity nodes
    '''
    for j in range(len(results_from_nlu)):
        for i in range(len(results_from_nlu[j]['entities'])):
            new_node_name = results_from_nlu[j]['entities'][i]['text']
            G.add_node(new_node_name)
            for k,v in results_from_nlu[j]['entities'][i].items():
                if( k != 'text'):
                    G.node[new_node_name][k]=v

def filter_and_format_relations(relationships):
    '''
    function to filter and format relations
    '''
    req_relations=[]
    filter_relations=[]
    relations= relationships
    for rel in relations:
        r= rel['type']
        score = rel['score']
        entity_name= list()
        entity_type= list()
        for arg in rel['arguments']:
            entity_name.append(arg['entities'][0]['text'])
            entity_type.append(arg['entities'][0]['type'])
        if((entity_type[0] == 'GeopoliticalEntity' or entity_type[1] == 'GeopoliticalEntity')):
            if(any(nodes_df['entity_names']== entity_name[0] ) and any(nodes_df['entity_names']== entity_name[1])):
                filter_relations.append(rel)
    
    
    for filter_rel in filter_relations:
        r= filter_rel['type']
        score = filter_rel['score']
        text= list()
        for arg in filter_rel['arguments']:
            text.append(arg['entities'][0]['text'])

        rel_tuple= list()
        rel_tuple.append(text[0])
        rel_tuple.append(r)
        rel_tuple.append(text[1])

        rel_tuple= tuple(rel_tuple)

        req_relations.append(rel_tuple)

        
    return req_relations
            
    
def draw_simple_graph(graph):
    '''
    funtion to draw graph
    '''
    nodes = []
    labels = []
    edges = []
    # extract nodes from graph
    for tuples in graph:
        nodes.append(tuples[0])
        nodes.append(tuples[2])
        
    # extract edges from graph
    for edgepairs in graph:
        edges.append((edgepairs[0],edgepairs[2]))        
    # extract edge labels from graph
    for edgetuples in graph:
        labels.append(edgetuples[1])
    # create networkx graph
    G=nx.Graph()
    # add nodes
    for node in nodes:
        G.add_node(node)
    # add edges
    for edge in graph:
        G.add_edge(edge[0], edge[2])

    # draw graph
    pos = nx.shell_layout(G)
    #print(pos)
    nx.draw(G, pos,with_labels = True)
    edge_labels = dict(zip(edges, labels))
    nx.draw_networkx_edge_labels(G, pos, edge_labels = edge_labels)

    # show graph
    plt.show()
    
    return G, pos, edge_labels   


def knowledge_graph(results_from_nlu):
    '''
    funtion to draw knowledge graph
    '''
    relationships=[]
    for i in range(len(results_from_nlu)):
        for j in range(len(results_from_nlu[i]['relations'])):
            relationships.append(results_from_nlu[i]['relations'][j])
    rel=[]
    configjson= json.loads(config_relation_json)
    for i in range(len(raw_texts)):
        res = extract_relations(raw_texts[i],configjson)
        if res:
            rel= res
    response=filter_and_format_relations(relationships)
    response= response + rel
    G, pos, edge_labels = draw_simple_graph(response)
    return G, pos, edge_labels


def disambiguate_entities(text):
    '''
    funtion to disambiguate entities
    '''
    keyword=[]
    for j in range(len(results_from_nlu)):
        for i in range(len(results_from_nlu[j]['keywords'])):
            for k,v in results_from_nlu[j]['keywords'][i].items():
                if(k=='text'):
                    keyword.append(v)
                
    for word in keyword:
        tag= '<Keyword:'
        if word in text:
            text=re.sub(word,tag+word+'>' ,text)
    return text

def extract_relations(text,configjson):
    '''
    funtion to extract relations
    '''
    relationship=[]
    relations= configjson['configuration']['relations']['rules']
    text=disambiguate_entities(text)
    for rel in relations:
        match= re.findall(rel['pattern'],text)
        if match:
            temp1= re.split('<Keyword:',match[0][0])
            temp2= re.split('<Keyword:',match[0][2])
            match1= re.split('>',temp1[1])
            match2= re.split('>',temp2[1])
            tuplerel=[]
            tuplerel.append(match1[0])
            tuplerel.append(rel['tag'])
            tuplerel.append(match2[0])
            relationship.append(tuple(tuplerel))
    return relationship

### 6. Process

In [ ]:
config_classification_json=load_string(get_file(credentials_2['FILE'])).decode("utf-8")

config_relation_json=load_string(get_file(credentials_3['FILE'])).decode("utf-8")

In [ ]:
results_from_nlu, results_from_htmlTable, raw_texts = generate_NLUJson_HtmlTablesJson()

In [ ]:
G = nx.MultiDiGraph()

In [ ]:
create_entityNodes(G,results_from_nlu)

In [ ]:

nodes_df = create_nodes_dataframe(G)

In [ ]:
G, pos, edge_labels = knowledge_graph(results_from_nlu)